# Modul 6
## Paulus Caesario Dito Putra Hartono
## 205314159

# Tugas 4

## Functions

In [ ]:
import numpy as np
import pandas as pd
from math import log

import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [ ]:
def removeNonWord(doc):
  result = doc.replace({'\w*\d\w*': ''}, regex=True)
  result = result.replace({'[\W_]+': ' '}, regex=True)
  return result[result.notnull()]

def docLength(doc):
  doc = doc.split()
  return len(doc)


def avdl(docs):
  docLengths = []
  total = len(docs)
  
  for i in range(len(docs)):
    docLengths.append(docLength(docs[i]))
  
  # print(docLengths)
  return np.sum(docLengths)/total


def count(word, doc):
  doc = doc.split()
  result = doc.count(word)
  return result


def idf(word, docs):
  dfw = 0
  for doc in docs:
    if doc.find(word) != -1:
      dfw += 1
  if dfw == 0:
    return 0
  result = log((len(docs) + 1)/ dfw, 10)
  return result


def tfWeight(word, doc, k):
  tfw = ((k+1)*count(word, doc))/(count(word,doc)+k)
  return tfw


def tfwpln(word, doc, k, b, d, avdl):
  tfwpln = ((k+1)*count(word,doc))/(count(word,doc)+k*(1-b+b*(abs(d)/avdl)))
  return tfwpln

In [ ]:
def calculateTF(queries, docs):
  tf = []
  filteredQueries = list(np.unique(queries))
  for d in range(len(docs)):
    tf.append([])
    for q in range(len(filteredQueries)):
      tf[d].append(count(filteredQueries[q], docs[d]))

  index = [x for x in range(len(docs))]
  df = pd.DataFrame(data=tf, index=index)
  df.columns=filteredQueries
  return df


def calculateIDF(queries, docs):
  idfs = []
  filteredQueries = list(np.unique(queries))
  for q in range(len(filteredQueries)):
    idfs.append(idf(filteredQueries[q], docs))

  index = [x for x in filteredQueries]
  df = pd.DataFrame(data=idfs, index=index)
  df.columns=['IDF']
  return df


def calculateTFW(queries, docs, k):
  fqd = []
  filteredQueries = list(np.unique(queries))
  for d in range(len(docs)):
    fqd.append([])
    for q in range(len(filteredQueries)):
      part1 = list(queries).count(filteredQueries[q])
      part2 = tfWeight(filteredQueries[q], docs[d], k)
      part3 = idf(filteredQueries[q], docs)
      fqd[d].append(part1*part2*part3)

  result = [sum(xs) for xs in fqd]
  index = ['f(q,d'+str(x+1)+')' for x in range(len(result))]
  df = pd.DataFrame(data=result, index=index)
  df.columns=['TFW']
  return df


def calculateTFWPLN(queries, docs, k, b):
  fqd = []
  filteredQueries = list(np.unique(queries))
  avgdocl = avdl(docs)
  for d in range(len(docs)):
    fqd.append([])
    for q in range(len(filteredQueries)):
      part1 = list(queries).count(filteredQueries[q])
      part2 = tfwpln(filteredQueries[q], docs[d], k, b, docLength(doc[d]), avgdocl)
      part3 = idf(filteredQueries[q], docs)
      fqd[d].append(part1*part2*part3)

  result = [sum(xs) for xs in fqd]
  index = ['f(q,d'+str(x+1)+')' for x in range(len(result))]
  df = pd.DataFrame(data=result, index=index)
  df.columns=['TFWPLN']
  return df

## Tugas

Tugas:
Sebuah mesin pencari ingin menemukan dokumen yang relevan dengan beberapa kata kunci dalam suatu Query. Temukan masing-masing tiga dokumen yang paling relevan dengan kata kunci dalam Query menggunakan pendekatan: 
1. Term Frequency Weight dengan parameter bobot k = 5; dan
2. Pivoted length normalization dengan parameter normalisasi b = 0.5. Untuk itu:

> a. Siapkan file doc.csv yang berisi 10 dokumen (kalimat singkat 1 paragraf) dengan topik sesuai pilihan anda

> b. Siapkan Query dengan beberapa kata kunci sesuai pilihan anda

> c. Temukan 3 dokumen yang relevan untuk masing-masing pendekatan.

> d. Hasil praktikum dikirim ke Asisten Dosen.

### a file doc.csv
sudah ada pada proses scraping sebelumnya (saya mengambil 11 document)

### Scraping ambil data

##### Unpreprocessed

In [ ]:
# https://jogja.tribunnews.com/2022/09/14/4-rekomendasi-kuliner-bubur-ayam-di-jogja-cocok-buat-menu-sarapan

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url = "https://jogja.tribunnews.com/2022/09/14/4-rekomendasi-kuliner-bubur-ayam-di-jogja-cocok-buat-menu-sarapan?page=all"

page=requests.get(url)
soup = bs(page.content, 'html.parser')

head_tag = soup.head

body_tag = soup.body

judul = head_tag.contents[1].contents[0]

# print('Judul: ', judul)

tanggal = body_tag.time.contents[0]

# print('Tgl Penulisan: ', tanggal)

img_thumb = body_tag.find_all('img', class_='imgfull')[0]['src']

# print('Url Thumbnail: ', img_thumb)

strong_str = body_tag.find_all('strong')

bolded_text = []
for x in strong_str:
  bolded_text.append(x.text)


paragraphs = body_tag.find_all('p')

paragraphs_content = []

for p in paragraphs:
  paragraphs_content.append(p.text)

# paragraphs_content

table = pd.DataFrame.from_dict({
  "Judul": judul,
  "Tanggal": tanggal,
  "Thumbnail": img_thumb,
  "Bolded-Tags": bolded_text,
  "Paragraphs": paragraphs_content
}, orient='index')

# table = table.transpose() 

table.to_csv('docUnpreprocessed.csv')

table

,0
Judul,4 Rekomendasi Kuliner Jogja Menyantap Bubur Ay...
Tanggal,"Rabu, 14 September 2022 11:16 WIB"
Thumbnail,https://t-2.tstatic.net/jogja/foto/bank/images...
Bolded-Tags,"[TRIBUNJOGJA.COM -, 1. Bubur Rempah DMR, 2. Bu..."
Paragraphs,[TRIBUNJOGJA.COM - Kuliner Jogja tidak akan ad...


In [ ]:
paragraphs_content

['TRIBUNJOGJA.COM - Kuliner\xa0Jogja tidak akan ada habisnya jika kita membicarakannya, mulai dari kuliner berat hingga kuliner ringan.',
 'Bubur menjadi salah satu spot makan Kuliner\xa0Jogja yang bisa kamu coba,',
 'Inilah rekomendasi makanan bubur ayam di Jogja untuk kamu cobain',
 '1. Bubur Rempah DMR',
 'Kuliner Jogja, bubur rempah DMR merupakan salah satu jenis restoran bubur ayam dengan rempah terkomplit pertama di Jogja.',
 'Bubur rempah DMR ini lokasinya ada di Jl. lpda Tutharsono no 3 (Delta Hotel) yang buka setiap senin - sabtu jam 08.00 -18.00',
 'Bubur rempah bisa juga diorder melalui GoFood, untuk harganya sendiri mulai 10k untuk porsi biasa dan 15k untuk porsai besarnya.',
 'Selain bubur disini kamu juga bisa memesan menu lainnya seperti siomay atau jenis minuman yang disediakan.',
 '2. Burnesia',
 'Buat pecinta bubur, burnesia merupakan outlet bubur terlengkap dengan berbagai macam jenis varian rasa.',
 'Di burnesia ini Kuliner\xa0Jogja dapat menikmati macam macam jenis

##### Preprocessed

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# print(paragraphs_content)

# data teks2
data = paragraphs_content

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
stopword = factory.create_stop_word_remover()

stemmed_words = []

for p in data:
  stemmed_words.append(stemmer.stem(p))

stemmed_words

preprocessed = []

for sentence in stemmed_words:
  preprocessed.append(stopword.remove(sentence))

preprocessed

table = pd.DataFrame({
    'sentences': preprocessed[0:11]
})

table.to_csv('doc.csv')

table

,sentences
0,tribunjogja com - kuliner jogja habis bicara k...
1,bubur salah spot makan kuliner jogja coba
2,rekomendasi makan bubur ayam jogja cobain
3,1 bubur rempah dmr
4,kuliner jogja bubur rempah dmr salah jenis res...
5,bubur rempah dmr lokasi jl lpda tutharsono no ...
6,bubur rempah order gofood harga 10k porsi 15k ...
7,bubur mes menu siomay jenis minum sedia
8,2 burnesia
9,cinta bubur burnesia outlet bubur lengkap jeni...


### b pembuatan wordbank

In [ ]:
# Pembuatan wordBank.csv

import pandas as pd

keywords = ['harga', 'buka', 'bubur', 'menu', 'lokasi', 'rempah', 'jenis', 'kuliner', 'jogja', 'murah']

wordBank = pd.DataFrame({
    'words': keywords
})

wordBank.to_csv('wordBank.csv')

wordBank

,words
0,harga
1,buka
2,bubur
3,menu
4,lokasi
5,rempah
6,jenis
7,kuliner
8,jogja
9,murah


### c menemukan 3 document yang relevan

In [ ]:
# k = 5 dan 2
# b = 0.5

doc = pd.read_csv('doc.csv', encoding='utf-8')
doc = doc['sentences'].str.lower()
doc = removeNonWord(doc)
vocabulary = pd.read_csv('wordBank.csv', encoding='utf-8')
vocabulary = vocabulary['words']

doc

0     tribunjogja com kuliner jogja habis bicara kul...
1             bubur salah spot makan kuliner jogja coba
2             rekomendasi makan bubur ayam jogja cobain
3                                      bubur rempah dmr
4     kuliner jogja bubur rempah dmr salah jenis res...
5     bubur rempah dmr lokasi jl lpda tutharsono no ...
6          bubur rempah order gofood harga porsi porsai
7               bubur mes menu siomay jenis minum sedia
8                                              burnesia
9     cinta bubur burnesia outlet bubur lengkap jeni...
10    burnesia kuliner jogja nikmat jenis bubur bubu...
Name: sentences, dtype: object

In [ ]:
vocabulary

0      harga
1       buka
2      bubur
3       menu
4     lokasi
5     rempah
6      jenis
7    kuliner
8      jogja
9      murah
Name: words, dtype: object

In [ ]:
# TF
result = calculateTF(vocabulary, doc)
result

,bubur,buka,harga,jenis,jogja,kuliner,lokasi,menu,murah,rempah
0,0,0,0,0,1,3,0,0,0,0
1,1,0,0,0,1,1,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1
4,2,0,0,1,2,1,0,0,0,2
5,1,1,0,0,0,0,1,0,0,1
6,1,0,1,0,0,0,0,0,0,1
7,1,0,0,1,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0
9,2,0,0,1,0,0,0,0,0,0


In [ ]:
# IDF
result = calculateIDF(vocabulary, doc)
result

,IDF
bubur,0.124939
buka,1.079181
harga,1.079181
jenis,0.477121
jogja,0.380211
kuliner,0.477121
lokasi,1.079181
menu,1.079181
murah,0.000000
rempah,0.477121


In [ ]:
# TFW k = 5
print('TFW k = 5')
result = calculateTFW(vocabulary, doc, 5)

result.sort_values(by='TFW', ascending=False)

TFW k = 5


,TFW
"f(q,d6)",2.760422
"f(q,d5)",2.638136
"f(q,d11)",1.771739
"f(q,d7)",1.681241
"f(q,d8)",1.681241
"f(q,d1)",1.453734
"f(q,d2)",0.982271
"f(q,d10)",0.691302
"f(q,d4)",0.602060
"f(q,d3)",0.505150


In [ ]:
# TFW k = 2
print('TFW k = 2')
result = calculateTFW(vocabulary, doc, 2)
# result

result.sort_values(by='TFW', ascending=False)

TFW k = 2


,TFW
"f(q,d6)",2.760422
"f(q,d5)",2.427649
"f(q,d7)",1.681241
"f(q,d8)",1.681241
"f(q,d11)",1.625977
"f(q,d1)",1.239030
"f(q,d2)",0.982271
"f(q,d10)",0.664529
"f(q,d4)",0.602060
"f(q,d3)",0.505150


In [ ]:
# PLN b = 0.5, k = 2 
print('PLN b = 0.5, k = 2')
result = calculateTFWPLN(vocabulary, doc, 2, 0.5)
# result

result.sort_values(by='TFWPLN', ascending=False)

PLN b = 0.5, k = 2


,TFWPLN
"f(q,d6)",2.349461
"f(q,d5)",2.172729
"f(q,d7)",1.825864
"f(q,d8)",1.825864
"f(q,d1)",1.213024
"f(q,d11)",1.092358
"f(q,d2)",1.066768
"f(q,d4)",0.776273
"f(q,d10)",0.692149
"f(q,d3)",0.571121


In [ ]:
# PLN b = 0.5, k = 5
print('PLN b = 0.5, k = 5')
result = calculateTFWPLN(vocabulary, doc, 5, 0.5)
# result

result.sort_values(by='TFWPLN', ascending=False)

PLN b = 0.5, k = 5


,TFWPLN
"f(q,d5)",2.279469
"f(q,d6)",2.265154
"f(q,d7)",1.865993
"f(q,d8)",1.865993
"f(q,d1)",1.411039
"f(q,d11)",1.098620
"f(q,d2)",1.090213
"f(q,d4)",0.836808
"f(q,d10)",0.728660
"f(q,d3)",0.590397


### Jawaban
Dari sorting jawaban diatas, dapat disimpulkan bahwa:
> ranking secara urut: d6, d5, d11